In [ ]:
# run against production 2025-03-11
from acdh_tei_pyutils.tei import TeiReader
from acdh_tei_pyutils.utils import get_xmlid
from acdh_xml_pyutils.xml import NSMAP
from normdata.utils import import_from_normdata
from tqdm import tqdm
import pandas as pd

In [ ]:
col, _ = Collection.objects.get_or_create(name="Stefan Zweig digital")
domain = "zweig-digital"
col_type, _ = CollectionType.objects.get_or_create(name="Projekt")
col.description = "https://www.stefanzweig.digital"
col.collection_type = col_type
col.save()

In [ ]:
doc = TeiReader("https://www.stefanzweig.digital/o:szd.personen/TEI_SOURCE")

In [ ]:
items = doc.any_xpath(".//tei:person[./tei:persName[@ref]]")
len(items)

In [ ]:
broken_gnd = []
pmb_uris = []
for x in tqdm(items, total=len(items)):
    xml_id = get_xmlid(x)
    domain_uri = f"https://www.stefanzweig.digital/o:szd.personen/sdef:TEI/get#{xml_id}"
    gnd_uri = x.xpath("./tei:persName/@ref", namespaces=NSMAP)[0].split()[0]
    entity = import_from_normdata(gnd_uri, "person")
    if entity:
        pmb_uri, _ = Uri.objects.get_or_create(uri=domain_uri, domain=domain)
        pmb_uri.entity = entity
        pmb_uri.save()
        entity.collection.add(col)
        pmb_uris.append([xml_id, entity.id])
    else:
        broken_gnd.append(gnd_uri)

In [ ]:
df = pd.DataFrame(pmb_uris)
df.to_csv('zweig_pmb.csv', index=False)

In [ ]:
df = pd.DataFrame(broken_gnd)
df.to_csv('zweig_broken_gnd.csv', index=False)